# PJ5
## 实验概述
- Task1：NEG
  **效果不如分类器模型，即使是GPT3.5**
  - Test1：采用Qwen/Qwen2.5-0.5B-Instruct进行提示词工程测试
  - Test2：采用OpenAI API进行提示词工程测试

- Task2：Summary
  
  **由于数据量过大，难以完全测试，总是会测试一半然后CUDA OOM，因此只测试了部分数据**
  **结果提高有限**
  - Test1：采用Qwen/Qwen2.5-0.5B-Instruct进行few-shot提示词测试
  - Test2：基于Qwen/Qwen2.5-0.5B-Instruct进行微调进行测试

## 定义数据集
* 定义Dataset：包含CHisIECDataset与 SummaryDataset


In [1]:
from torch.utils.data import Dataset
from typing import List, Optional
import torch
import torch.nn as nn
import json
class CHisIECDataset(Dataset):
    label_label_id_mapping = {
        "O": 0,
        "B-PER": 1,
        "I-PER": 2,
        "E-PER": 3,
        "S-PER": 4,
        "B-LOC": 5,
        "I-LOC": 6,
        "E-LOC": 7,
        "S-LOC": 8,
        "B-OFI": 9,
        "I-OFI": 10,
        "E-OFI": 11,
        "S-OFI": 12,
        "B-BOOK": 13,
        "I-BOOK": 14,
        "E-BOOK": 15,
        "S-BOOK": 16,
    }
    
    def __init__(self, path) -> None:
        super().__init__()
        self.data = []
        with open(path, "r", encoding="utf-8") as f:
            d = ['', '']
            while line := f.readline():
                line = line.strip()
                if line:
                    word, label = line.split()
                    d[0]+=word
                    
                    d[1]+=' ' + label+  ' '
                elif d[0]:
                    self.data.append(tuple(d))
                    d = ['', '']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]
    
class SummaryDataset(Dataset):
    
    def __init__(self, path):
        super().__init__()
        self.data = []
        # 解析 JSON 数据
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                    # 解析每一行的 JSON 对象
                    item = json.loads(line.strip())
                    
                    # 提取所需字段
                    article = item['article']
                    summary = item['summary']
                    id = item['id']
                    label = item['label']
                    
                    # 将解析的数据添加到 self.data 列表中
                    self.data.append({
                        'article': article,
                        'summary': summary,
                        'id': id,
                        'label': label
                    })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

* 加载数据集

In [3]:

from torch.utils.data import DataLoader
from torch.nn.functional import one_hot

## 定义get_dataloader创建Dataload,用于处理从数据集中获取的批次数据
## batchsize（？）
def get_CHisIECtest_dataloader(dataset, shuffle=True):
    def collect_fn(batch):
        t = batch[0][0]
        l = batch[0][1]
        return t, l

    return DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=1,
        collate_fn=collect_fn,
    )

def get_Summary_dataloader(dataset, shuffle=True):
    def collect_fn(batch):
        t = batch[0]['article']
        l = batch[0]['summary']
        return t, l

    return DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=1,
        collate_fn=collect_fn,
    )
   
CHisIECtest_set = CHisIECDataset("ner.txt")
Summary_set = SummaryDataset("summary.jsonl")

CHisIECtest_loader = get_CHisIECtest_dataloader(CHisIECtest_set, shuffle=False)
#输出元素
# for i, (data, label) in enumerate(CHisIECtest_loader):
#     print(data)
#     print(label)
    
Summarytest_loader = get_Summary_dataloader(Summary_set,False)
# for i, (data, label) in enumerate(Summarytest_loader):
    # print(data)
    # print(label)   

## 模型测试




In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re


# 创建一个设备对象
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"当前设备: {device}")
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
cache_directory = "F:/model"  # 指定你想要的缓存目录

# 加载模型和分词器
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map=device,
    cache_dir=cache_directory  # 添加此行
)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_directory,device_map=device)  # 添加此行

当前设备: cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### NER
最终结果都很差 很难高于0.1
#### Test 1
* 采用不同形式的prompt，多次尝试
  * zero-shot ： 只给任务提示
    结果很差，乱生成
  * few-shot ： 给出两个示例
    结果变好，但是部分样本会受到示例的影响，结果与样本结果相似
    但是最后的F1-SCORE 很低 因为大部分都在生成 O

* 结论：黑盒很难去调整，最后效果很大取决于预训练模型的能力

    

In [ ]:


all_true_labels = []
all_pred_labels = []
# 假设你已经定义了 test_loader
prompt_task = "我需要你去做实体标记任务 给以上序列的每个词打上标注,输出长度应该与句子长度相同  "

for i, (data, label) in enumerate(CHisIECtest_set):
    print(i)
    q =  data + prompt_task  # 将数据添加到 prompt 中
    messages = [
        # {"role": "system", "content":  },
        {"role": "user", "content": '''以下我需要你做实体标记任务,为每个字打上以下标签,以下是标签的介绍:B-PER(个人名称实体的开始), I-PER(个人名称实体的内部), E-PER(个人名称实体的结束), S-PER(单独的个人名称), B-LOC(地点实体的开始), I-LOC(地点实体的内部), E-LOC(地点实体的结束), S-LOC(单独的地点), B-OFI(组织机构实体的开始), I-OFI(组织机构实体的内部), E-OFI(组织机构实体的结束), S-OFI(单独的组织机构), B-BOOK(书名实体的开始), I-BOOK(书名实体的内部), E-BOOK(书名实体的结束), S-BOOK(单独的书名) 
        训旣作相，以守澄为六军十二卫观军容使，罢其禁旅之权，寻赐酖杀之。训愈承恩顾，每别殿奏对，他宰相莫不顺成其言，黄门禁军迎拜戢敛。'''+prompt_task+'''
    Answer : S-PER  O  O  S-OFI  O  O  B-PER  E-PER  O  B-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  E-OFI  O  O  O  O  O  O  O  O  O  O  O  O  O  O  S-PER  O  O  O  O  O  O  O  O  O  O  O  O  B-OFI  E-OFI  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
    赜与洛阳元善、河东柳[巩言，凡改几]、太原王劭、吴兴姚察、琅邪诸葛颍、信都刘焯、河间刘炫相善，每因休假，清谈竟日。'''+prompt_task+'''
    Answer :  S-PER  O  B-LOC  E-LOC  B-PER  E-PER  O  B-LOC  E-LOC  S-PER  O  O  O  O  O  O  O  O  O  B-LOC  E-LOC  B-PER  E-PER  O  B-LOC  E-LOC  B-PER  E-PER  O  B-LOC  E-LOC  B-PER  I-PER  E-PER  O  B-LOC  E-LOC  B-PER  E-PER  O  B-LOC  E-LOC  B-PER  E-PER  O  O  O  O  O  O  O  O  O  O  O  O  O 
    '''+q+'''\nAnswer:'''}
    ]

    # 将消息格式化为模型输入
    text = tokenizer(messages[0]["content"], return_tensors="pt").to(device)

    # 生成输出
    generated_ids = model.generate(
        **text,
        max_new_tokens=512
    )

    # 解码生成的文本
    response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # 使用正则表达式找到所有 'Answer' 后的字符组
    matches = re.findall(r'Answer:\s*(.*?)(?=Answer:|$)', response, re.DOTALL)

    # 提取最后一个匹配
    answer_text = matches[-1].strip()    
    pred_labels = answer_text.split('  ')  # 将模型输出分割成标签列表
    
    # 真实标签
    true_labels = label.split('  ')  # 真实标签从字符串转换为列表
    if len(pred_labels) > len(true_labels):
        pred_labels = pred_labels[:len(true_labels)]
    elif len(pred_labels) < len(true_labels):
        
        pred_labels += ["None"] * (len(true_labels) - len(pred_labels))
    pred_labels = [label.strip() for label in pred_labels if label.strip()]
    true_labels = [label.strip() for label in true_labels if label.strip()]
    # 存储到全局列表
    all_true_labels.extend(true_labels)
    all_pred_labels.extend(pred_labels)
    





0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216


In [12]:
## fewshot
from sklearn.metrics import f1_score
f1 = f1_score(all_true_labels, all_pred_labels, average='macro')  # 或者选择'micro' 或 'macro'作为 average 参数
print(f"F1 Score: {f1:.4f}")


F1 Score: 0.0239


#### Test 2
**使用API进行测试，基于更大模型**
prompt: 
* 1:直接用句子做fewshot 输出格式正确 但是F1准确率低下，稳定在0.1以内
  ```py
  q ='''以下我需要你做实体标记任务,为每个字打上以下标签,以下是标签的介绍:B-PER(个人名称实体的开始), I-PER(个人名称实体的内部), E-PER(个人名称实体的结束), S-PER(单独的个人名称), B-LOC(地点实体的开始), I-LOC(地点实体的内部), E-LOC(地点实体的结束), S-LOC(单独的地点), B-OFI(组织机构实体的开始), I-OFI(组织机构实体的内部), E-OFI(组织机构实体的结束), S-OFI(单独的组织机构), B-BOOK(书名实体的开始), I-BOOK(书名实体的内部), E-BOOK(书名实体的结束), S-BOOK(单独的书名) 
    
    
    Sentence:训旣作相，以守澄为六军十二卫观军容使，罢其禁旅之权，寻赐酖杀之。训愈承恩顾，每别殿奏对，他宰相莫不顺成其言，黄门禁军迎拜戢敛。'''+'''
    Answer : S-PER  O  O  S-OFI  O  O  B-PER  E-PER  O  B-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  E-OFI  O  O  O  O  O  O  O  O  O  O  O  O  O  O  S-PER  O  O  O  O  O  O  O  O  O  O  O  O  B-OFI  E-OFI  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
    '''+q+'''\nAnswer :'''
  ```
* 2：尝试对每个标签做出用例解释，之后在做用句子做fewshot 
  * 效果更差了，可能由于样例与文本不够契合
```py
'''以下我需要你做实体标记任务,为每个字打上以下标签,以下是标签的介绍:
    B-PER：表示个人名称实体的开始。例如，“李”在“李华”中是名字的开始。
    I-PER：表示个人名称实体的内部部分。例如，“华”在“李华”中是名字的中间部分。
    E-PER：表示个人名称实体的结束。例如，在“王小明”中，“明”就是这个个人名称实体的结尾。
    S-PER：表示单独的个人名称。例如，“李”如果作为一个独立的名字出现。
    B-LOC：表示地点实体的开始。例如，“北”在“北京”中是地点名称的开始。
    I-LOC：表示地点实体的内部部分。例如，“京”在“北京”中是地点名称的内部。
    E-LOC：表示地点实体的结束。例如，在“上海市”中，“市”就是这个地点名称的结尾。
    S-LOC：表示单独的地点。例如，“京”如果作为单独的地点出现。
    B-ORG：表示组织机构实体的开始。例如，“中”在“中国科学院”中是机构名称的开始。
    I-ORG：表示组织机构实体的内部部分。例如，“国”在“中国科学院”中是机构名称的内部。
    E-ORG：表示组织机构实体的结束。例如，在“中国科学院”中，“院”就是这个机构名称的结尾。
    S-ORG：表示单独的组织机构。例如，“中科院”如果作为单独的简称出现。
    B-BOOK：表示书名实体的开始。例如，“三”在“三国演义”中是书名的开始。
    I-BOOK：表示书名实体的内部部分。例如，“国”在“三国演义”中是书名的内部。
    E-BOOK：表示书名实体的结束。例如，在“三国演义”中，“义”就是这个书名的结尾。
    S-BOOK：表示单独的书名。例如，“经”如果作为一个独立的书名出现
    
    Sentence:训旣作相，以守澄为六军十二卫观军容使，罢其禁旅之权，寻赐酖杀之。训愈承恩顾，每别殿奏对，他宰相莫不顺成其言，黄门禁军迎拜戢敛。'''+'''
    Answer : S-PER  O  O  S-OFI  O  O  B-PER  E-PER  O  B-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  E-OFI  O  O  O  O  O  O  O  O  O  O  O  O  O  O  S-PER  O  O  O  O  O  O  O  O  O  O  O  O  B-OFI  E-OFI  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
    '''+q+'''\nAnswer :'''
```

In [9]:
import requests
all_true_labels = []
all_pred_labels = []
# 假设你已经定义了 test_loader
prompt_task = "Sentence: "
url = "https://www.dmxapi.com/v1/chat/completions"

for i, (data, label) in enumerate(CHisIECtest_set):
    print(i)
    q =  data + prompt_task  # 将数据添加到 prompt 中
    q ='''以下我需要你做实体标记任务,为每个字打上以下标签,以下是标签的介绍:B-PER(个人名称实体的开始), I-PER(个人名称实体的内部), E-PER(个人名称实体的结束), S-PER(单独的个人名称), B-LOC(地点实体的开始), I-LOC(地点实体的内部), E-LOC(地点实体的结束), S-LOC(单独的地点), B-OFI(组织机构实体的开始), I-OFI(组织机构实体的内部), E-OFI(组织机构实体的结束), S-OFI(单独的组织机构), B-BOOK(书名实体的开始), I-BOOK(书名实体的内部), E-BOOK(书名实体的结束), S-BOOK(单独的书名) 
    
    
    Sentence:训旣作相，以守澄为六军十二卫观军容使，罢其禁旅之权，寻赐酖杀之。训愈承恩顾，每别殿奏对，他宰相莫不顺成其言，黄门禁军迎拜戢敛。'''+'''
    Answer : S-PER  O  O  S-OFI  O  O  B-PER  E-PER  O  B-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  E-OFI  O  O  O  O  O  O  O  O  O  O  O  O  O  O  S-PER  O  O  O  O  O  O  O  O  O  O  O  O  B-OFI  E-OFI  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
    '''+q+'''\nAnswer :'''
    # q ='''以下我需要你做实体标记任务,为每个字打上以下标签,以下是标签的介绍:
    # B-PER：表示个人名称实体的开始。例如，“李”在“李华”中是名字的开始。
    # I-PER：表示个人名称实体的内部部分。例如，“华”在“李华”中是名字的中间部分。
    # E-PER：表示个人名称实体的结束。例如，在“王小明”中，“明”就是这个个人名称实体的结尾。
    # S-PER：表示单独的个人名称。例如，“李”如果作为一个独立的名字出现。
    # B-LOC：表示地点实体的开始。例如，“北”在“北京”中是地点名称的开始。
    # I-LOC：表示地点实体的内部部分。例如，“京”在“北京”中是地点名称的内部。
    # E-LOC：表示地点实体的结束。例如，在“上海市”中，“市”就是这个地点名称的结尾。
    # S-LOC：表示单独的地点。例如，“京”如果作为单独的地点出现。
    # B-ORG：表示组织机构实体的开始。例如，“中”在“中国科学院”中是机构名称的开始。
    # I-ORG：表示组织机构实体的内部部分。例如，“国”在“中国科学院”中是机构名称的内部。
    # E-ORG：表示组织机构实体的结束。例如，在“中国科学院”中，“院”就是这个机构名称的结尾。
    # S-ORG：表示单独的组织机构。例如，“中科院”如果作为单独的简称出现。
    # B-BOOK：表示书名实体的开始。例如，“三”在“三国演义”中是书名的开始。
    # I-BOOK：表示书名实体的内部部分。例如，“国”在“三国演义”中是书名的内部。
    # E-BOOK：表示书名实体的结束。例如，在“三国演义”中，“义”就是这个书名的结尾。
    # S-BOOK：表示单独的书名。例如，“经”如果作为一个独立的书名出现
    
    # Sentence:训旣作相，以守澄为六军十二卫观军容使，罢其禁旅之权，寻赐酖杀之。训愈承恩顾，每别殿奏对，他宰相莫不顺成其言，黄门禁军迎拜戢敛。'''+'''
    # Answer : S-PER  O  O  S-OFI  O  O  B-PER  E-PER  O  B-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  I-OFI  E-OFI  O  O  O  O  O  O  O  O  O  O  O  O  O  O  S-PER  O  O  O  O  O  O  O  O  O  O  O  O  B-OFI  E-OFI  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
    # '''+q+'''\nAnswer :'''
    payload = json.dumps({
                "model": "gpt-3.5-turbo",  # 这里是你需要访问的模型，改成上面你需要测试的模型名称就可以了。
                "messages": [
                    {
                        "role": "system",
                        "content": "你是命名实体识别任务的模型，只需要输出标签，以空格分隔，标点符号也作为识别的对象"
                    },
                    {
                        "role": "user",
                        "content": q
                    }
                ]
            })
    headers = {
                'Accept': 'application/json',
                'Authorization': 'sk-kxRQ7ArIeazjscgawqVHPMCE2MJqYpM7LdT2RcMYlE7sq5i9',  # 这里放你的 DMXapi key
                'User-Agent': 'DMXAPI/1.0.0 (https://www.dmxapi.com)',  # 这里也改成 DMXAPI 的中转URL https://www.dmxapi.com，已经改好
                'Content-Type': 'application/json'
            }
    response = requests.request("POST", url, headers=headers, data=payload)
    response_json = json.loads(response.text)
    reply = response_json['choices'][0]['message']['content']  # 这里是返回结果
   
    
    
    pred_labels = reply.split(' ')  # 将模型输出分割成标签列表
    
    # 真实标签
    true_labels = label.split('  ')  # 真实标签从字符串转换为列表
    if len(pred_labels) > len(true_labels):
        pred_labels = pred_labels[:len(true_labels)]
    elif len(pred_labels) < len(true_labels):
        pred_labels += ["None"] * (len(true_labels) - len(pred_labels))
    # 存储到全局列表
    all_true_labels.extend(true_labels)
    all_pred_labels.extend(pred_labels)
    print(len(all_true_labels))
    print(len(all_pred_labels))
    # if(i==100):
    #     break
    from sklearn.metrics import f1_score
    f1 = f1_score(all_true_labels, all_pred_labels, average='macro')  # 或者选择'micro' 或 'macro'作为 average 参数
    print(f"F1 Score: {f1:.4f}")
    

0
93
93
F1 Score: 0.0452
1
144
144
F1 Score: 0.0386
2
210
210
F1 Score: 0.0361
3
266
266
F1 Score: 0.0352
4
307
307
F1 Score: 0.0349
5
317
317
F1 Score: 0.0478
6
384
384
F1 Score: 0.0498
7
459
459
F1 Score: 0.0433
8
519
519
F1 Score: 0.0412
9
534
534
F1 Score: 0.0384
10
628
628
F1 Score: 0.0407
11
668
668
F1 Score: 0.0416
12
756
756
F1 Score: 0.0450
13
774
774
F1 Score: 0.0512
14
800
800
F1 Score: 0.0501
15
865
865
F1 Score: 0.0488
16
954
954
F1 Score: 0.0514
17
992
992
F1 Score: 0.0494
18
1064
1064
F1 Score: 0.0459
19
1117
1117
F1 Score: 0.0446
20
1188
1188
F1 Score: 0.0441
21
1262
1262
F1 Score: 0.0432
22
1295
1295
F1 Score: 0.0427
23
1325
1325
F1 Score: 0.0423
24
1408
1408
F1 Score: 0.0415
25
1425
1425
F1 Score: 0.0471
26
1495
1495
F1 Score: 0.0458
27
1536
1536
F1 Score: 0.0450
28
1609
1609
F1 Score: 0.0454
29
1645
1645
F1 Score: 0.0464
30
1682
1682
F1 Score: 0.0454
31
1771
1771
F1 Score: 0.0453
32
1847
1847
F1 Score: 0.0465
33
1924
1924
F1 Score: 0.0449
34
1994
1994
F1 Score: 0.044

KeyError: 'choices'

In [10]:
## fewshot gpt
from sklearn.metrics import f1_score
print(len(all_true_labels))
print(len(all_pred_labels))
f1 = f1_score(all_true_labels, all_pred_labels, average='macro')  # 或者选择'micro' 或 'macro'作为 average 参数
print(f"F1 Score: {f1:.4f}")

12884
12884
F1 Score: 0.0344


### Summary
#### Test1：qwen

**测试过程中总会出现cuda out of memory的错误**

**测试输出结果具有随机性，容易匹配不到内容，这和模型能力有关，GPT3.5就输出格式很规范**
* 采用不同prompt测试
  * zero-shot
    会输出概括，但是相对较长
  * few-shot
    输出长度更短，指标更高
    

In [7]:

prompt_task = '''我需要你做中文概括任务，以下是一个示例
原句 : 24岁 的 男子 小罗 ( 化名 ) 因 患有 精神 病 被 送进 医院 , 为 防止 他 伤害 自己 , 医院 用 保护 带 将 其 绑 在床上 。第二天 , 护士 发现 小罗 被 同室 病人 掐死 。", "为此 , 小罗 母亲 提起 刑事 自诉 , 要求 追究 当班 护士 的 刑事责任 。", "法 晚 记者 上午 获悉 , 市 一中 院 终审 判处 当班 护士 杨某 犯 医疗 事故 罪 , 但 由于 医院 已 与 家属 达成 协议 , 赔偿 了 50万 元 , 且 杨某 已 被 医院 开除 , 故 对 其 免予 刑事 处罚 。", "住院 当晚 被 同室 病人 掐死 1999年 11月 29 日 , 24岁 的 被害人 小罗 因 精神 受到 刺激 被 送往 本市 海淀区 北京大学 第六 医院 治疗 。", "经 诊断 系 “ 急性 精神病 症 ” , 医嘱 为 精神科 特护 ( 防 冲动 、 巡视 )", "当晚 , 因 小罗 拒绝 服药 打针 , 医院 用 保护 带 将 其 绑 在床上 , 后 又 因其 吵闹 , 医师 给 其 注射 了 安定 10mg 。", "次日 早晨 6 点 10分 , 小罗 被 发现 已 死亡 。", "海淀 法院 一审 判决书 查明 , 1999年 11月 30 日 , 杨某 在 本市 海淀区 北京 医科 大学 附属 第六 医院 住院部 , 作为 大夜班 副 班 护士 , 未 严格 履行 巡视 职责 , 对 已 约束 的 病人 未 按 规定 定时 松解 保护 带 。", "导致 小罗 在 杨某 值班 期间 , 被 同室 39 岁 的 精神病人 唐某 扼 颈 机械 性窒息 死亡 。", "经 司法 精神病 鉴定 , 唐某 实施 违法行为 时 丧失 辨认 、 控制 能力 , 无责任 能力 。", "2000年 1 月 19日 , 北京 医科 大学 附属 第六 医院 与 被害人 家属 达成 协议 , 赔偿 被害人 家属 50万 元 并 承担 其他 损失 费用 。", "医院 对 护士 杨某 和 郝某 予以 行政 处分 , 决定 对 杨某 给予 开除 行政 处分 。", "当班 护士 认为 “ 应 由 医院 担责 ” 杨某 供述 称 , 1999年 11月 29 日 下午 1 时 许 , 护士 站 急 收 了 病人 小罗 , 有 3 名 家属 护送 。", "当时 , 小罗 比较 兴奋 吵闹 , 杨某 就 腾出 一个 单间 安排 其 入住 , 并 对 小罗 采取 了 保护 措施 , 用 绑带 把 他 绑 在 病床 上 。", "把 门锁 上后 , 杨某 于 当日 下午 3 时 就 下班 了 。", "晚上 11点 多 , 杨某 到 单位 上夜班 。", "到了 次日 凌晨 1点 30 分 , 他 和 另外 一 名 实习 护士 郝某 与 前 班 护士 办理 交接 。", "杨某 首先 隔着 门 玻璃 查看 清点 了 一下 人数 , 郝某 清点 医用 物品 , 后 一起 听取 了 前 一班 护士 的 交班 报告 。", "在 清点 人数 时 , 杨某 发现 小罗 的 病房 里 又新 进 了 一个 病人 唐某 。", "当晚 , 杨某 在 交接班 之后 , 只 在 凌晨 2 点 左右 巡视 过 1 次 , 只是 清点 了 人数 , 没有 进 病房 查看 , 之后 就 再 没有 巡视 查看 过 。", "次日 早上 6 点 多钟 , 郝某 找到 杨某 说 小罗 有些 不对劲 。", "杨某 到 小罗 床 前 掀开 被子 , 发现 小罗 还 被 捆着 , 身体 已经 僵 了 。", "杨某 认为 自己 虽有 过错 , 但 尚 不 构成 犯罪 , 受害人 小罗 的 死亡 结果 应当 由 医院 承担 责任 。", "杨某 辩护人 认为 , 受害人 死亡 的 原因 是 多 方面 的 , 既有 唐某 的 直接 原因 , 也 有 医院 管理 混乱 , 工作 交接 脱节 , 上 一班 医护 人员 错误 安排 病房 、 没有 及时 松解 保护 带 等 方面 原因 。", "杨某 未 按 规定 巡视 并不 必然 导致 被害人 小罗 死亡 的 结果 发生 。", "家属 向 护士 提起 刑事 自诉 郝某 称 , 按规定 , 应该 对 病人 每隔 10分钟 巡视 1 次 , 对 小罗 这样 的 病人 应该 重点 观察 。", "此外 , 对于 小罗 这样 有 被 约束 病人 的 病房 里 , 是 不能 再 入住 没有 被 约束 的 病人 的 。", "医院 出具 的 特护 记录 显示 , 杨某 的 前 一班 护士 对 小罗 都 有 详细 且 明确 的 护理 记录 , 但 从 当晚 1 点 半 杨某 接班 以后 , 就 没有 任何 关于 对 小罗 的 护理 记录 , 杨某 一 晚上 都 没有 对 小罗 进行 过 巡查 , 也 没有 按规定 护理 。", "2006年 , 北京 医学会 、 中华 医学会 两次 进行 医疗 事故 鉴定 , 结论 均 为 : 本 例 医疗 事故 争议 属于 一级 甲等 医疗 事故 , 医院 在 患者 的 损伤 结果 中 承担 次要 责任 。", "检察机关 认定 被告人 杨某 涉嫌 医疗 事故 罪 , 但 情节 轻微 , 决定 对 被告人 杨某 不起诉 。", "为此 , 小罗 的 母亲 赵某 向 海淀 法院 提起 刑事 自诉 。", "构成 医疗 事故 罪 免予 刑事 处罚 海淀 法院 认为 , 多项 证据 显示 , 虽然 被害人 小罗 的 死亡 是 由 第三人 病态 行为 直接 造成 , 但 该 死亡 结果 是 在 被害人 就诊 期间 发生 , 由于 医院 管理 不当 和 严重 失职 所 导致 的 。", "杨某 身为 当晚 值班 副 班 护士 , 是 事发 当时 代表 医院 承担 巡视 、 护理 职责 , 其 严重 不 负责 的 行为 是 医院 没有 履行 好 保护 患者 安全 职责 的 表现 之一 , 与 该 死亡 结果 存在 着 重要 的 因果关系 , 应当 以 医疗 事故 罪 追究 其 刑事责任 。", "鉴于 此次 医疗 事故 系 由 多 原因 所 造成 , 医院 多 方面 的 过失 并非 仅限于 杨某 单个 因素 。", "故 杨某 虽 构成 犯罪 , 但 情节 相对 轻微 , 且 系 初犯 , 到案 后 能 如实 供认 其 失职 行为 , 具有 悔罪 表现 , 同时 事后 杨某 已 被 医院 处以 开除 的 行政 处分 , 受到 相应 惩罚 , 已 无 判处 刑罚 之 必要 , 故 法院 认为 可以 对 其 免予 刑事 处罚 。", "据此 , 海淀 法院 作出 一审 判决 , 杨某 犯 医疗 事故 罪 , 免予 刑事 处罚 。", "一审 判决 后 , 小罗 家属 提起 上诉 , 认为 原判 对 杨某 的 处罚 过轻 。", "杨某 也 提起 了 上诉 , 杨某 认为 被害人 的 死亡 后果 并非 其 造成 的 , 其 不 构成 犯罪 。", "市 一中 院 审理 后 , 驳回 杨某 和 赵 女士 的 上诉 , 维持 原判 。"
概括 : 北京 精神病人 被 用 束缚 带 绑 床上 , 遭 病友 掐死 ; 法院 认定 医院 管理 不当 、 值班 护士 失职 , 构成 医疗 事故 罪 。<END>
用简短的语句概括以下内容,概括的句子 <END> 结尾
原句 :'''
import re
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
from modelscope import AutoModelForCausalLM, AutoTokenizer
batch_size = 8  # 设置批处理大小（可以根据GPU的内存进行调整）
all_answer = []
all_pre = []

from modelscope import Model
from swift import Swift
import torch
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.bfloat16, 
    device_map='auto', 
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# 将数据分成小批次
for i in range(0, len(Summary_set), batch_size):
    batch_data = Summary_set[i:i+batch_size]
    print(f"Processing batch {i//batch_size + 1}...")

    # 准备输入数据
    batch_q = [prompt_task + ''.join(data['article']) + "\n概括 :" for data in batch_data]
    batch_a = [data['summary'] for data in batch_data]
    # print(batch_q)
    # 将批量数据转化为模型输入
    inputs = tokenizer(batch_q, padding=True, truncation=True, return_tensors="pt").to('cuda')
    
    # 生成输出
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=128
    )

    # 解码生成的文本
    responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
   
    
    # 处理每个生成的文本
    for j, response in enumerate(responses):
        matches = re.findall(r"概括 :(.*?)\<END\>", response, re.DOTALL)
        if len(matches) > 1:
            all_pre.append(matches[1])
            all_answer.append(batch_a[j])
        else:
            print(f"第 {i + j} 个数据未匹配到足够的内容，重新提问...")








    
    

[INFO:modelscope] Creating symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct.
[WARNING:modelscope] Failed to create symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct: [Errno 17] File exists: '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct' -> '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct'


[INFO:modelscope] Creating symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct.
[WARNING:modelscope] Failed to create symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct: [Errno 17] File exists: '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct' -> '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct'
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processing batch 1...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


第 1 个数据未匹配到足够的内容，重新提问...
第 2 个数据未匹配到足够的内容，重新提问...
第 3 个数据未匹配到足够的内容，重新提问...
第 6 个数据未匹配到足够的内容，重新提问...
第 7 个数据未匹配到足够的内容，重新提问...
Processing batch 2...


...

第 1969 个数据未匹配到足够的内容，重新提问...
第 1971 个数据未匹配到足够的内容，重新提问...
第 1972 个数据未匹配到足够的内容，重新提问...
第 1973 个数据未匹配到足够的内容，重新提问...
第 1975 个数据未匹配到足够的内容，重新提问...
Processing batch 248...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


第 1977 个数据未匹配到足够的内容，重新提问...
第 1980 个数据未匹配到足够的内容，重新提问...
第 1982 个数据未匹配到足够的内容，重新提问...
Processing batch 249...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


第 1984 个数据未匹配到足够的内容，重新提问...
第 1985 个数据未匹配到足够的内容，重新提问...
第 1987 个数据未匹配到足够的内容，重新提问...
第 1988 个数据未匹配到足够的内容，重新提问...
第 1989 个数据未匹配到足够的内容，重新提问...
第 1990 个数据未匹配到足够的内容，重新提问...
第 1991 个数据未匹配到足够的内容，重新提问...
Processing batch 250...
第 1992 个数据未匹配到足够的内容，重新提问...
第 1993 个数据未匹配到足够的内容，重新提问...
第 1994 个数据未匹配到足够的内容，重新提问...
第 1997 个数据未匹配到足够的内容，重新提问...
第 1998 个数据未匹配到足够的内容，重新提问...
第 1999 个数据未匹配到足够的内容，重新提问...


In [ ]:

with open('predictions_test1.json', 'w') as f:
    json.dump(all_pre, f)

with open('references_test1.json', 'w') as f:
    json.dump(all_answer, f)


In [5]:
import json
import evaluate

# 从文件中读取数据
with open('predictions_test1.json', 'r') as f:
    all_pre = json.load(f)

with open('references_test1.json', 'r') as f:
    all_answer = json.load(f)

# 加载 ROUGE 评估器
rouge = evaluate.load('rouge')

# 确保 all_pre 和 all_answer 已包含所有预测和真实摘要
# all_pre.append(matches[0])

# 计算 ROUGE 分数
print(len(all_answer))
results = rouge.compute(predictions=all_pre, references=all_answer)

# 输出结果
print("ROUGE Scores:")
print(f"ROUGE-1: {results['rouge1']:.4f}")
print(f"ROUGE-2: {results['rouge2']:.4f}")
print(f"ROUGE-L: {results['rougeL']:.4f}")

    

746
ROUGE Scores:
ROUGE-1: 0.0695
ROUGE-2: 0.0099
ROUGE-L: 0.0690


#### Test 2
**微调预训练模型**

基于魔搭平台的modelscope库，使用预训练模型进行微调，并使用自定义数据集进行模型评估。
##### 微调数据集构造

In [1]:
import json


path = 'data\\test2017.simple.label.jsonl'
prompt_task = '''\n我需要你做中文概括任务。用简短的语句概括以下内容,概括的句子 <END> 结尾
原句 :'''
# 用于存储转换后的内容
converted_data = []

# 读取并转换 jsonl 文件
with open(path, 'r', encoding='utf-8') as f:
    for line in f:
        # 解析每一行的 JSON 对象
        item = json.loads(line.strip())
                    
        # 提取所需字段
        article = item['article']
        artical = ''.join(article)
        summary = item['summary']
        summary = ''.join(summary)
        
        q = prompt_task + artical + "\n概括 :"
        a = "概括 :" + summary            
        
        converted_data.append({'query': q, 'answer': a})

output_file_path = 'data\CNewSum_v2\\final\\fineture.jsonl'
# 输出转换后的内容
with open(output_file_path, 'w', encoding='utf-8') as f:
    for item in converted_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')  

##### 微调过程
**不使用全量微调，采用LoRA微调降低对设备的要求**

**数据来自CNewSum_v2的其他test集合**


**回答格式与提示词要求符合度有提高**

**最后结果提高有限，可能由于微调数据集有限，同时构建较为粗糙**


* 微调

In [10]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, MsDataset
from swift import Seq2SeqTrainer, Seq2SeqTrainingArguments, Swift, LoRAConfig
from swift.llm import get_template, TemplateType
import json
import pandas as pd
import torch

model_name = "Qwen/Qwen2.5-0.5B-Instruct"
file_path = 'fineture.jsonl'

# 数据加载函数
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                record = json.loads(line.strip())
                data.append(record)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid line: {line}, error: {e}")
    return data

def save_as_csv(data, csv_file_path):
    df = pd.DataFrame(data)
    df.to_csv(csv_file_path, index=False, encoding='utf-8')

def create_msdataset_from_csv(csv_file_path):
    return MsDataset.load(csv_file_path)

# 数据预处理
csv_file_path = 'output.csv'
data = load_jsonl(file_path)
save_as_csv(data, csv_file_path)
dataset = create_msdataset_from_csv(csv_file_path)

# 加载模型和 tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.bfloat16, 
    device_map='auto', 
    trust_remote_code=True
)
lora_config = LoRAConfig(
    r=16,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_alpha=32,
    lora_dropout=0.05
)
model = Swift.prepare_model(model, lora_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 模板设置
template = get_template(TemplateType.chatglm3, tokenizer, max_length=1024)

# 数据编码函数
def encode(example):
    question = example.get('question', '')
    answer = example.get('answer', '')
    if not answer:
        return None
    example, kwargs = template.encode({'query': question, 'response': answer})
    return example

# 手动映射数据
encoded_data = []
for record in dataset:
    try:
        encoded_example = encode(record)
        if encoded_example:  # 确保不为空
            encoded_data.append(encoded_example)
    except Exception as e:
        print(f"Error encoding record {record}: {e}")

# 保存处理后的数据为新的 CSV
processed_csv_file = 'processed_dataset.csv'
pd.DataFrame(encoded_data).to_csv(processed_csv_file, index=False, encoding='utf-8')

# 从新的 CSV 文件加载 MsDataset

# 数据集划分
# dataset_split = encoded_dataset.train_test_split(test_size=0.001)
# train_dataset, val_dataset = dataset_split['train'], dataset_split['test']

# 训练参数
train_args = Seq2SeqTrainingArguments(
    output_dir='output',
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    eval_steps=50,
    save_steps=50,
    logging_steps=10,
    evaluation_strategy='no',
    save_strategy='steps',
    save_total_limit=2,
    dataloader_num_workers=4,
    warmup_steps=100
)

# 创建 Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=train_args,
    data_collator=template.data_collator,
    train_dataset=encoded_data,
    tokenizer=tokenizer
)

# 开始训练
trainer.train()



Generating train split: 0 examples [00:00, ? examples/s]

[INFO:modelscope] Creating symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct.
[WARNING:modelscope] Failed to create symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct: [Errno 17] File exists: '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct' -> '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct'


[INFO:modelscope] Creating symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct.
[WARNING:modelscope] Failed to create symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct: [Errno 17] File exists: '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct' -> '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct'
/usr/local/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/site-packages/swift/trainers/mixin.py:93: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
Detected kernel 

Train:   0%|          | 0/375 [00:00<?, ?it/s]

{'loss': 3.16273766, 'acc': 0.36791363, 'grad_norm': 2.265625, 'learning_rate': 1e-05, 'memory(GiB)': 3.89, 'train_speed(iter/s)': 0.635844, 'epoch': 0.08, 'global_step/max_steps': '10/375', 'percentage': '2.67%', 'elapsed_time': '15s', 'remaining_time': '9m 17s'}
{'loss': 2.66615562, 'acc': 0.40595317, 'grad_norm': 1.5234375, 'learning_rate': 2e-05, 'memory(GiB)': 2.74, 'train_speed(iter/s)': 0.657411, 'epoch': 0.16, 'global_step/max_steps': '20/375', 'percentage': '5.33%', 'elapsed_time': '29s', 'remaining_time': '8m 51s'}
{'loss': 2.37051659, 'acc': 0.44810405, 'grad_norm': 1.1015625, 'learning_rate': 3e-05, 'memory(GiB)': 2.74, 'train_speed(iter/s)': 0.660203, 'epoch': 0.24, 'global_step/max_steps': '30/375', 'percentage': '8.00%', 'elapsed_time': '44s', 'remaining_time': '8m 37s'}
{'loss': 2.3178812, 'acc': 0.4581409, 'grad_norm': 1.1875, 'learning_rate': 4e-05, 'memory(GiB)': 2.95, 'train_speed(iter/s)': 0.663176, 'epoch': 0.32, 'global_step/max_steps': '40/375', 'percentage': '1

...

[INFO:swift] Saving model checkpoint to output/checkpoint-350


{'loss': 1.69155102, 'acc': 0.58140535, 'grad_norm': 1.28125, 'learning_rate': 5.45e-06, 'memory(GiB)': 2.95, 'train_speed(iter/s)': 0.666877, 'epoch': 2.88, 'global_step/max_steps': '360/375', 'percentage': '96.00%', 'elapsed_time': '8m 59s', 'remaining_time': '22s'}
{'loss': 1.83493042, 'acc': 0.55252657, 'grad_norm': 1.2890625, 'learning_rate': 1.82e-06, 'memory(GiB)': 2.95, 'train_speed(iter/s)': 0.667086, 'epoch': 2.96, 'global_step/max_steps': '370/375', 'percentage': '98.67%', 'elapsed_time': '9m 14s', 'remaining_time': '7s'}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[INFO:swift] Saving model checkpoint to output/checkpoint-375


{'train_runtime': 562.2168, 'train_samples_per_second': 10.672, 'train_steps_per_second': 0.667, 'train_loss': 2.05023495, 'epoch': 3.0, 'global_step/max_steps': '375/375', 'percentage': '100.00%', 'elapsed_time': '9m 22s', 'remaining_time': '0s'}


TrainOutput(global_step=375, training_loss=2.0502349497477215, metrics={'train_runtime': 562.2168, 'train_samples_per_second': 10.672, 'train_steps_per_second': 0.667, 'total_flos': 1078855936012800.0, 'train_loss': 2.05023495, 'epoch': 3.0, 'global_step/max_steps': '375/375', 'percentage': '100.00%', 'elapsed_time': '9m 22s', 'remaining_time': '0s'})

In [16]:
output_directory = 'modified_model'
model.save_pretrained(output_directory)

* 测试

In [33]:
prompt_task = '''我需要你做中文概括任务，以下是一个示例
原句 : 24岁 的 男子 小罗 ( 化名 ) 因 患有 精神 病 被 送进 医院 , 为 防止 他 伤害 自己 , 医院 用 保护 带 将 其 绑 在床上 。第二天 , 护士 发现 小罗 被 同室 病人 掐死 。", "为此 , 小罗 母亲 提起 刑事 自诉 , 要求 追究 当班 护士 的 刑事责任 。", "法 晚 记者 上午 获悉 , 市 一中 院 终审 判处 当班 护士 杨某 犯 医疗 事故 罪 , 但 由于 医院 已 与 家属 达成 协议 , 赔偿 了 50万 元 , 且 杨某 已 被 医院 开除 , 故 对 其 免予 刑事 处罚 。", "住院 当晚 被 同室 病人 掐死 1999年 11月 29 日 , 24岁 的 被害人 小罗 因 精神 受到 刺激 被 送往 本市 海淀区 北京大学 第六 医院 治疗 。", "经 诊断 系 “ 急性 精神病 症 ” , 医嘱 为 精神科 特护 ( 防 冲动 、 巡视 )", "当晚 , 因 小罗 拒绝 服药 打针 , 医院 用 保护 带 将 其 绑 在床上 , 后 又 因其 吵闹 , 医师 给 其 注射 了 安定 10mg 。", "次日 早晨 6 点 10分 , 小罗 被 发现 已 死亡 。", "海淀 法院 一审 判决书 查明 , 1999年 11月 30 日 , 杨某 在 本市 海淀区 北京 医科 大学 附属 第六 医院 住院部 , 作为 大夜班 副 班 护士 , 未 严格 履行 巡视 职责 , 对 已 约束 的 病人 未 按 规定 定时 松解 保护 带 。", "导致 小罗 在 杨某 值班 期间 , 被 同室 39 岁 的 精神病人 唐某 扼 颈 机械 性窒息 死亡 。", "经 司法 精神病 鉴定 , 唐某 实施 违法行为 时 丧失 辨认 、 控制 能力 , 无责任 能力 。", "2000年 1 月 19日 , 北京 医科 大学 附属 第六 医院 与 被害人 家属 达成 协议 , 赔偿 被害人 家属 50万 元 并 承担 其他 损失 费用 。", "医院 对 护士 杨某 和 郝某 予以 行政 处分 , 决定 对 杨某 给予 开除 行政 处分 。", "当班 护士 认为 “ 应 由 医院 担责 ” 杨某 供述 称 , 1999年 11月 29 日 下午 1 时 许 , 护士 站 急 收 了 病人 小罗 , 有 3 名 家属 护送 。", "当时 , 小罗 比较 兴奋 吵闹 , 杨某 就 腾出 一个 单间 安排 其 入住 , 并 对 小罗 采取 了 保护 措施 , 用 绑带 把 他 绑 在 病床 上 。", "把 门锁 上后 , 杨某 于 当日 下午 3 时 就 下班 了 。", "晚上 11点 多 , 杨某 到 单位 上夜班 。", "到了 次日 凌晨 1点 30 分 , 他 和 另外 一 名 实习 护士 郝某 与 前 班 护士 办理 交接 。", "杨某 首先 隔着 门 玻璃 查看 清点 了 一下 人数 , 郝某 清点 医用 物品 , 后 一起 听取 了 前 一班 护士 的 交班 报告 。", "在 清点 人数 时 , 杨某 发现 小罗 的 病房 里 又新 进 了 一个 病人 唐某 。", "当晚 , 杨某 在 交接班 之后 , 只 在 凌晨 2 点 左右 巡视 过 1 次 , 只是 清点 了 人数 , 没有 进 病房 查看 , 之后 就 再 没有 巡视 查看 过 。", "次日 早上 6 点 多钟 , 郝某 找到 杨某 说 小罗 有些 不对劲 。", "杨某 到 小罗 床 前 掀开 被子 , 发现 小罗 还 被 捆着 , 身体 已经 僵 了 。", "杨某 认为 自己 虽有 过错 , 但 尚 不 构成 犯罪 , 受害人 小罗 的 死亡 结果 应当 由 医院 承担 责任 。", "杨某 辩护人 认为 , 受害人 死亡 的 原因 是 多 方面 的 , 既有 唐某 的 直接 原因 , 也 有 医院 管理 混乱 , 工作 交接 脱节 , 上 一班 医护 人员 错误 安排 病房 、 没有 及时 松解 保护 带 等 方面 原因 。", "杨某 未 按 规定 巡视 并不 必然 导致 被害人 小罗 死亡 的 结果 发生 。", "家属 向 护士 提起 刑事 自诉 郝某 称 , 按规定 , 应该 对 病人 每隔 10分钟 巡视 1 次 , 对 小罗 这样 的 病人 应该 重点 观察 。", "此外 , 对于 小罗 这样 有 被 约束 病人 的 病房 里 , 是 不能 再 入住 没有 被 约束 的 病人 的 。", "医院 出具 的 特护 记录 显示 , 杨某 的 前 一班 护士 对 小罗 都 有 详细 且 明确 的 护理 记录 , 但 从 当晚 1 点 半 杨某 接班 以后 , 就 没有 任何 关于 对 小罗 的 护理 记录 , 杨某 一 晚上 都 没有 对 小罗 进行 过 巡查 , 也 没有 按规定 护理 。", "2006年 , 北京 医学会 、 中华 医学会 两次 进行 医疗 事故 鉴定 , 结论 均 为 : 本 例 医疗 事故 争议 属于 一级 甲等 医疗 事故 , 医院 在 患者 的 损伤 结果 中 承担 次要 责任 。", "检察机关 认定 被告人 杨某 涉嫌 医疗 事故 罪 , 但 情节 轻微 , 决定 对 被告人 杨某 不起诉 。", "为此 , 小罗 的 母亲 赵某 向 海淀 法院 提起 刑事 自诉 。", "构成 医疗 事故 罪 免予 刑事 处罚 海淀 法院 认为 , 多项 证据 显示 , 虽然 被害人 小罗 的 死亡 是 由 第三人 病态 行为 直接 造成 , 但 该 死亡 结果 是 在 被害人 就诊 期间 发生 , 由于 医院 管理 不当 和 严重 失职 所 导致 的 。", "杨某 身为 当晚 值班 副 班 护士 , 是 事发 当时 代表 医院 承担 巡视 、 护理 职责 , 其 严重 不 负责 的 行为 是 医院 没有 履行 好 保护 患者 安全 职责 的 表现 之一 , 与 该 死亡 结果 存在 着 重要 的 因果关系 , 应当 以 医疗 事故 罪 追究 其 刑事责任 。", "鉴于 此次 医疗 事故 系 由 多 原因 所 造成 , 医院 多 方面 的 过失 并非 仅限于 杨某 单个 因素 。", "故 杨某 虽 构成 犯罪 , 但 情节 相对 轻微 , 且 系 初犯 , 到案 后 能 如实 供认 其 失职 行为 , 具有 悔罪 表现 , 同时 事后 杨某 已 被 医院 处以 开除 的 行政 处分 , 受到 相应 惩罚 , 已 无 判处 刑罚 之 必要 , 故 法院 认为 可以 对 其 免予 刑事 处罚 。", "据此 , 海淀 法院 作出 一审 判决 , 杨某 犯 医疗 事故 罪 , 免予 刑事 处罚 。", "一审 判决 后 , 小罗 家属 提起 上诉 , 认为 原判 对 杨某 的 处罚 过轻 。", "杨某 也 提起 了 上诉 , 杨某 认为 被害人 的 死亡 后果 并非 其 造成 的 , 其 不 构成 犯罪 。", "市 一中 院 审理 后 , 驳回 杨某 和 赵 女士 的 上诉 , 维持 原判 。"
概括 : 北京 精神病人 被 用 束缚 带 绑 床上 , 遭 病友 掐死 ; 法院 认定 医院 管理 不当 、 值班 护士 失职 , 构成 医疗 事故 罪 。<END>
用简短的语句概括以下内容,概括的句子 <END> 结尾
原句 :'''
import re
batch_size = 8  # 设置批处理大小（可以根据GPU的内存进行调整）
all_answer = []
all_pre = []
output_model_dir = 'modified_model'
from modelscope import Model
from swift import Swift
import torch
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.bfloat16, 
    device_map='auto', 
    trust_remote_code=True
)
model = Swift.from_pretrained(model, './output')

# 将数据分成小批次
for i in range(0, len(Summary_set), batch_size):
    batch_data = Summary_set[i:i+batch_size]
    print(f"Processing batch {i//batch_size + 1}...")

    # 准备输入数据
    batch_q = [prompt_task + ''.join(data['article']) + "\n概括 :" for data in batch_data]
    batch_a = [data['summary'] for data in batch_data]
    # print(batch_q)
    # 将批量数据转化为模型输入
    inputs = tokenizer(batch_q, padding=True, truncation=True, return_tensors="pt").to('cuda')
    
    # 生成输出
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=128
    )

    # 解码生成的文本
    responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
   
    
    # 处理每个生成的文本
    for j, response in enumerate(responses):
        matches = re.findall(r"概括 :(.*?)\<END\>", response, re.DOTALL)
        if len(matches) > 1:
            all_pre.append(matches[1])
            all_answer.append(batch_a[j])
        else:
            print(f"第 {i + j} 个数据未匹配到足够的内容，重新提问...")

    

        

[INFO:modelscope] Creating symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct.
[WARNING:modelscope] Failed to create symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct: [Errno 17] File exists: '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct' -> '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct'
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processing batch 1...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


第 0 个数据未匹配到足够的内容，重新提问...
第 2 个数据未匹配到足够的内容，重新提问...
第 3 个数据未匹配到足够的内容，重新提问...
第 4 个数据未匹配到足够的内容，重新提问...
第 5 个数据未匹配到足够的内容，重新提问...
第 6 个数据未匹配到足够的内容，重新提问...
Processing batch 2...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


第 1490 个数据未匹配到足够的内容，重新提问...
第 1491 个数据未匹配到足够的内容，重新提问...
第 1492 个数据未匹配到足够的内容，重新提问...
第 1495 个数据未匹配到足够的内容，重新提问...
Processing batch 188...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


第 1497 个数据未匹配到足够的内容，重新提问...
第 1498 个数据未匹配到足够的内容，重新提问...
第 1499 个数据未匹配到足够的内容，重新提问...
第 1500 个数据未匹配到足够的内容，重新提问...
第 1501 个数据未匹配到足够的内容，重新提问...
第 1502 个数据未匹配到足够的内容，重新提问...
第 1503 个数据未匹配到足够的内容，重新提问...
Processing batch 189...


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.34 GiB. GPU 

In [1]:
import json
import evaluate

# 从文件中读取数据
with open('predictions.json', 'r') as f:
    all_pre = json.load(f)

with open('references.json', 'r') as f:
    all_answer = json.load(f)

# 确保数据已经正确读取
print(f"Number of predictions: {len(all_pre)}")
print(f"Number of references: {len(all_answer)}")

# 加载 ROUGE 评估器
rouge = evaluate.load('rouge')

# 计算 ROUGE 分数
results = rouge.compute(predictions=all_pre, references=all_answer)

# 输出结果
print("ROUGE Scores:")
print(f"ROUGE-1: {results['rouge1']:.4f}")
print(f"ROUGE-2: {results['rouge2']:.4f}")
print(f"ROUGE-L: {results['rougeL']:.4f}")

Number of predictions: 584
Number of references: 584
ROUGE Scores:
ROUGE-1: 0.0526
ROUGE-2: 0.0113
ROUGE-L: 0.0535


### 神经网络模型与LLM在NER任务上的对比
本质上：一个是分类器 一个是生成模型
在训练过程中
* LLM F1-macro  
  * 数次实验都低于 0.1；调整prompt并没有起到很好的效果；很大取决于模型能力与提示词的引导
  * 在预训练模型上，只需要少量数据，就可以达到一定效果,但整体效果不如BiLSTM模型
  * 泛化能力强，自身携带大量的知识，但是基于Prompt输出效果的提高具有随机性，是一个黑盒
  * 需要确定最后的输出内容，然后通过数据处理手段去提取生成的答案
  * 生成答案的长度也有可能与序列不一致，需要去调整，这是大模型本身就存在的问题，对数字不敏感
* BiLSTM模型 F1-macro 0.7
  * 需要大量数据训练
  * 在固定任务上，有明确的优化目标，更能够去调试模型
  * 结果明确，可以直接使用

根据先前的实验结果，有以下对比：

| 特性                    | BiLSTM                      | LLM + Prompt                  |
|-----------------------|--------------------------------------------|--------------------------------------------|
| **模型结构**          | Glove + BiLSTM      **分类模型**            | Qwen  **生成式模型**      |
| **输入数据处理**      | 需要标签化数据，配合词嵌入        |依赖 tokenizer 将文本转为 token 序列，再引导生成基于上下文的输出        |
| **特征提取方式**      | 预训练嵌入向量，已经LSTM提取语义             | 自动从上下文中提取特征，无需手动特征设计      |
| **训练复杂度**        | 较低，参数量相对较少，易于调参               | 非常高，参数量大，依赖大规模计算资源        |
| **训练数据需求**      | 需要大量标签数据以提高效果                   | 可以通过few-shot并配合 prompt 调整效果，少量数据更强   |
| **推理速度**          | 快速，适合实时应用                           | 较慢，尤其是在大型模型情况下                 |
| **适应性**            | 针对任务训练，较难泛化到其他任务              | 通过 prompt 调整，具备一定的跨任务泛化能力   |
| **性能**              | 表现稳定，准确性依赖于标注数据和训练         | 数据稀少情况下，由于自身的知识更多，比LSTM表现好，整体上准确率提高太依赖prompt了，有随机性       |
| **输出控制**          | 明确控制输出，通过模型结构和损失函数调节       | 通过 prompt 控制输出，灵活性较高但难于完全控制；需要最后通过数据处理手段去提取答案 |
| **解释性**            | 结构清晰，易于理解输出和调试                 | 黑盒性强，输出依赖模型内部推断过程           |
| **资源需求**          | 资源需求相对较低                             | 资源需求大，需要 GPU 等计算设备支持         |



### 总结
#### 结果
* NER 
  * Test1
    F1 Score: 0.0239
  * Test2
    F1 Score: 0.0344
    
* Summary
  * Test1
    ROUGE Scores:
    ROUGE-1: 0.0695
    ROUGE-2: 0.0099
    ROUGE-L: 0.0690
  * Test2
    ROUGE Scores:
    ROUGE-1: 0.0526
    ROUGE-2: 0.0113
    ROUGE-L: 0.0535
#### 想法
* 预训练模型能力很大决定了任务的效果上限，GPT3.5输出的结果明显比Qwen更符合要求
* prompt只能起到锦上添花的作用，总不能让奔波儿灞去除掉唐僧师徒吧；GPT3.5都干不好的活，Qwen 0.5b....
* prompt可能会起到反作用 ，例如NER任务的例子第一个tag是S-PRE，那么很容易的结果的第一个就会被引导成S-PRE
* 微调模型需要更为细致的数据集分配，粗糙的数据集可以让模型输出内容更符合要求，但质量不一定能提高
* 训练模型需要的资源太大了，Cuda OOM 真的让人痛苦
* 对固定的NLP任务，传统方法是具有一定优势的，而大模型泛化能力更强
  